# 1) Importazione librerie

In [65]:
import scipy as sci
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from PIL import Image

from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras import saving

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, confusion_matrix, mean_squared_error

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

import gc

import ImageFilter as filter

# 2) Caricamento dataset
Le variabili _whistleImagesDirectory, noiseImagesDirectory_ contengono rispettivamente l'indirizzo della locazione in memoria dei file appartenenti alla classe dei delfini (1) e la classe dei rumori(0).

Prima vengono caricate le immagini dei fischi, successivamente quelle dei rumori. In entrambi i casi, ogni file viene caricato e ridimensionato nelle dimensioni (224,224,1), quindi sono immagini 2D.

I file di entrambe le classi vengono aggiunte alla lista _images_, mentre alla lista _labels_ vengono aggiunte le etichette in formato numerico. Per le immagini che rappresentano un delfino l'etichetta è 1, mentre i rumori hanno l'etichetta 0.

Poi, tutte le immagini vengono normalizzate, quindi i valori dei pixel non hanno più un valore tra 0 e 255, ma tra 0 e 1.

Successivamente, è possibile suddividere il dataset in train, validation (opzionale) e test nelle percentuali desiderate.

Al momento dell'addestramento della rete neurale (più avanti), si può scegliere di utilizzare la Fold Cross-Validation, in cui viene effettuata una suddivisione del dataset in train e test differente per ogni fold. Per cui, in tal caso non è necessario eseguire la suddivisione del dataset in questo blocco di codice perchè viene effettuata dopo.

In [87]:
# Directory containing images
whistleImagesDirectory = 'Data\\Sobel\\Whistle'
noiseImagesDirectory = 'Data\\Sobel\\Noise'

# List all image files in the directory
whistleFiles = os.listdir(whistleImagesDirectory)
noiseFiles = os.listdir(noiseImagesDirectory)

# Initialize empty lists for images and labels
images = []
labels = []

# Load and preprocess whistle images
for image_file in whistleFiles:
    image_path = os.path.join(whistleImagesDirectory, image_file)
    image = Image.open(image_path).convert('L')  # Convert to grayscale
    image = image.resize((224, 224))  # Resize
    image = np.array(image)  # Convert to NumPy array
    images.append(image)
    labels.append(1)    #etichetta delfino


# Load and preprocess noise images
for image_file in noiseFiles:
    image_path = os.path.join(noiseImagesDirectory, image_file)
    image = Image.open(image_path).convert('L')  # Convert to grayscale
    image = image.resize((224, 224))
    image = np.array(image)
    images.append(image)
    labels.append(0)    #etichetta rumore


X = np.array(images)/255.0
y = np.array(labels)

# Split the dataset into training and test sets                 (90/10)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_val, y_val = [], []

# Split the dataset into training, validation, and test sets    (70/20/10)
#X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
#X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.33, random_state=42)

# Print the shapes of the resulting sets
print("Train set shapes:", X_train.shape, y_train.shape)
#print("Validation set shapes:", X_val.shape, y_val.shape)
print("Test set shapes:", X_test.shape, y_test.shape)

Train set shapes: (1367, 224, 224) (1367,)
Test set shapes: (152, 224, 224) (152,)


In [43]:
#Verifica normalizzazione, il valore deve essere contenuto tra 0 e 1.
print(np.max(X))

1.0

Visualizzazione informazione dataset, riguardo al numero di fischi e di rumori presenti nel dataset intero e in quelli di train, validation e test.

In [55]:
totW = 0
totN = 0

def printAndSaveData(nameDivision, whistle, noise):
    print(f'Whistle in {nameDivision}: {whistle}')
    print(f'Noise in {nameDivision}: {noise}\n')


def getStatsDataset(y_true):
    nWhistle = 0
    nNoise = 0
    y_true = np.array(y_true)
    nWhistle += np.count_nonzero(y_true == 1)
    nNoise += np.count_nonzero(y_true == 0)
    return nWhistle, nNoise


nWhistle, nNoise = getStatsDataset(y)
printAndSaveData('total dataset', nWhistle, nNoise)

nWhistle, nNoise = getStatsDataset(y_train)
totW += nWhistle
totN += nNoise
printAndSaveData('dataset train', nWhistle, nNoise)

nWhistle, nNoise = getStatsDataset(y_val)
totW += nWhistle
totN += nNoise
printAndSaveData('dataset validation', nWhistle, nNoise)

nWhistle, nNoise = getStatsDataset(y_test)
totW += nWhistle
totN += nNoise
printAndSaveData('dataset test', nWhistle, nNoise)

Whistle in total dataset: 596
Noise in total dataset: 923

Whistle in dataset train: 530
Noise in dataset train: 837

Whistle in dataset validation: 0
Noise in dataset validation: 0

Whistle in dataset test: 66
Noise in dataset test: 86



# 3) Architettura modello

In [57]:
def getModel():
    model = tf.keras.models.Sequential([

    #Input Layer
    Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 1)),
    MaxPooling2D(2, 2),

    #Secondo Layer
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    #Terzo Layer
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    #Quarto Layer
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    #Quinto Layer
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),

    #Primo Dense Layer
    Dense(512, activation='relu'),

    #Output Dense Layer
    Dense(1, activation='sigmoid')])

    return model

In [64]:
#Visualizzazione numero di parametri del modello
model = getModel()
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_14 (Conv2D)          (None, 222, 222, 32)      320       
                                                                 
 max_pooling2d_14 (MaxPooli  (None, 111, 111, 32)      0         
 ng2D)                                                           
                                                                 
 conv2d_15 (Conv2D)          (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_15 (MaxPooli  (None, 54, 54, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_16 (Conv2D)          (None, 52, 52, 64)        36928     
                                                                 
 max_pooling2d_16 (MaxPooli  (None, 26, 26, 64)       

# 4) Addestramento modello
Esistono diversi tipi di addestramento. Vengono riportati i principali, ovvero quello normale e quello con la Fold Cross-Validation

### 4.1) Addestramento senza Fold Cross-Validation
Utilizzo ottimizzatore "Adam" con learning rate 0.0001. Con un learning rate più piccolo, la rete ha bisogno di più epoche per imparare. La funzione di loss impostata è "BinaryCrossentropy", ideale per problemi di classificazione binaria. L'accuracy è la metrica scelta.

Per addestrare il modello, bisogna scegliere un numero di epoche. Poche epoche non permettono alla rete di imparare bene, troppe epoche possono invece causare l'overfitting. Per cui è stato utilizzato l'EarlyStopping che lascia addestrare la modello fino ad un numero massimo di epoche settato a 100, monitorando in questo caso il valore di "val_loss" con pazienza 15. Se dopo 15 epoche di fila il valore di "val_loss" non migliora, l'addestramento viene interrotto.

Viene inoltre settato il batch size a 32.

L'attributo class_weights viene utilizzato per fare in modo che le due classi siano "sbilanciate". Con questi valori, la classe dei rumori ha più peso rispetto alla classe dei delfini. Ciò è stato fatto in quanto si preferisce avere dei falsi negativi piuttosto che dei falsi positivi.

Utilizzare anche l'attributo "validation_data" solo se parte del dataset è stato suddiviso anche per la validazione.


In [ ]:
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])


class_weights = {0: 1.5,  # noise
                 1: 1.0}  # dolphin's whistle


early_stop = EarlyStopping(monitor='val_loss', patience=15, verbose=1)
hist = model.fit(X_train, y_train, batch_size=32, epochs=100, class_weight=class_weights, callbacks=[early_stop])
#hist = model.fit(X_train, y_train, batch_size=32, epochs=100, validation_data=(X_val, y_val),
#                 class_weight=class_weights, callbacks=[early_stop])

### 4.2) Fold Cross-Validation
In questa parte di codice, viene effettuata una 5 Fold Cross-Validation.

#### 4.2.1) Addestramento modelli
Il dataset intero viene suddiviso in 5 parti (20% ciascuna). Vengono addestrati 5 modelli della stessa architettura, ma con un dataset differente. Pre ogni modello viene utilizzata una delle 5 fold differente come dataset di test, le altre invece per il train. L'addestramento per tutti i modelli è lo stesso riportato al blocco di codice 4.1). Dopo l'addestramento, con una soglia di 0.5 vengono calcolati Precision, Recall, Accuracy, F1-Score e MSE. Inoltre, viene calcolata anche la Confusion Matrix che riporta il numero di TP,TN,FP,FN.
Ricapitolando, tutti i modelli sono stati addestrati con il dataset suddiviso in 80% train e 20% test, ma il materiale utilizzato è diverso per ogni modello.

In [5]:
totFold = 5

kf = KFold(n_splits=totFold, shuffle=True, random_state=42)

arrHist = []
models = []

precisions = []
recalls = []
accuracies = []
f1_scores = []

cms = []
mse_scores = []

train_indexes = []
test_indexes = []

fold_num = 1
KFoldSplitted = kf.split(X)

for train_index, test_index in KFoldSplitted:
    print(f"Training on fold {fold_num}/{totFold}")

    print(len(train_index), len(test_index))
    train_indexes.append(train_index)
    test_indexes.append(test_index)

    # Splitting the data for this fold
    X_train_fold, X_test_fold = X[train_index], X[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]


    model = getModel()

    optimizer = Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer, loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

    early_stop = EarlyStopping(monitor='loss', patience=15, verbose=1)

    class_weights = {0: 1.5,  # noise
                     1: 1.0}  # dolphin's whistle

    # Continue training the model
    hist = model.fit(X_train_fold, y_train_fold, batch_size=32, epochs=200, class_weight=class_weights, callbacks=[early_stop])

    arrHist.append(hist)
    models.append(model)


    y_pred = model.predict(X_test_fold)
    y_pred_binary = (y_pred > 0.5).astype(np.int32)  # Convert probabilities to binary predictions

    # Calculate precision and recall
    precisions.append(precision_score(y_test_fold, y_pred_binary))
    recalls.append(recall_score(y_test_fold, y_pred_binary))
    accuracies.append(accuracy_score(y_test_fold, y_pred_binary))
    f1_scores.append(f1_score(y_test_fold, y_pred_binary))

    cm = confusion_matrix(y_test_fold, y_pred_binary)
    cms.append(cm)
    TN, FP, FN, TP = cm.ravel()

    # Calculate the mean squared error
    mse = mean_squared_error(y_test_fold, y_pred_binary)
    mse_scores.append(mse)

    print('\n\n\n')

    print('Precision: ', precisions[fold_num-1])
    print('Recall: ', recalls[fold_num-1])
    print('Accuracy: ', accuracies[fold_num-1])
    print('F1-Score: ', f1_scores[fold_num-1])
    print()
    print(f"True Positives (TP): {TP}")
    print(f"True Negatives (TN): {TN}")
    print(f"False Positives (FP): {FP}")
    print(f"False Negatives (FN): {FN}")
    print()
    print(f"Mean Squared Error: {mse}")


    # Increment the fold number for the next iteration
    fold_num += 1

Training on fold 1/5
1215 304
Epoch 1/200
38/38 [==============================] - 40s 1s/step - loss: 0.8177 - accuracy: 0.6082
Epoch 2/200
38/38 [==============================] - 36s 950ms/step - loss: 0.8027 - accuracy: 0.6082
Epoch 3/200
38/38 [==============================] - 32s 834ms/step - loss: 0.7990 - accuracy: 0.6082
Epoch 4/200
38/38 [==============================] - 32s 850ms/step - loss: 0.7998 - accuracy: 0.6082
Epoch 5/200
38/38 [==============================] - 33s 860ms/step - loss: 0.8017 - accuracy: 0.6082
Epoch 6/200
38/38 [==============================] - 31s 821ms/step - loss: 0.8031 - accuracy: 0.6082
Epoch 7/200
38/38 [==============================] - 31s 819ms/step - loss: 0.7981 - accuracy: 0.6082
Epoch 8/200
38/38 [==============================] - 33s 861ms/step - loss: 0.7933 - accuracy: 0.6082
Epoch 9/200
38/38 [==============================] - 33s 878ms/step - loss: 0.7919 - accuracy: 0.6082
Epoch 10/200
38/38 [==============================] - 3

# 5) Valutazione modello

### 5.1) Valutazione rete senza fold cross validation
Inserendo una soglia a 0.5, vengono processate dalla rete neurale tutte le immagini presenti nel dataset di test. Dopodichè vengono calcolate le metriche di valutazione Precision, Recall, Accuracy, F1-Score e MSE. Inoltre, viene calcolata anche la Confusion Matrix che riporta il numero di TP,TN,FP,FN.


In [ ]:
# Make predictions on the test set
y_pred = model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(np.int32)  # Convert probabilities to binary predictions

# Calculate precision and recall
precision = precision_score(y_test, y_pred_binary)
recall = recall_score(y_test, y_pred_binary)
accuracy = accuracy_score(y_test, y_pred_binary)
f1 = f1_score(y_test, y_pred_binary)

# Print precision and recall accuracy f1 score
print("Precision:", precision)
print("Recall:", recall)
print("Accuracy:", accuracy)
print("F1-Score:", f1)

cm = confusion_matrix(y_test, y_pred_binary)
TN, FP, FN, TP = cm.ravel()
cm = (np.array(cm))
print(cm)

print(f"True Positives (TP): {TP}")
print(f"True Negatives (TN): {TN}")
print(f"False Positives (FP): {FP}")
print(f"False Negatives (FN): {FN}")

### 5.2) Valutazione rete con fold cross validation

#### 5.2.1) Visualizzazione media valori
Per ottenere dei valori onesti delle metriche di valutazione di una rete neurale citate sopra, è opportuno calcolarne la media dei valori di tutti e 5 i modelli.

In [6]:
averagePrecision = 0
averageRecall = 0
averageAccuracy = 0
averageF1 = 0
averageMse = 0

for i in range(totFold):
    averagePrecision += precisions[i]
    averageRecall += recalls[i]
    averageAccuracy += accuracies[i]
    averageF1 += f1_scores[i]
    averageMse += mse_scores[i]

averagePrecision /= totFold
averageRecall /= totFold
averageAccuracy /= totFold
averageF1 /= totFold
averageMse /= totFold

print('Average Precision: ', averagePrecision)
print('Average Recall: ',averageRecall)
print('Average Accuracy: ', averageAccuracy)
print('Average F1-Score: ', averageF1)
print('Average Mean Squared Error: ', averageMse)


Average Precision:  0.9945775729646698
Average Recall:  0.9780364455121433
Average Accuracy:  0.9894606565919751
Average F1-Score:  0.9861790095890546
Average Mean Squared Error:  0.010539343408025013


#### 5.2.2) Visualizzazione valori di tutti i modelli
In questo blocco di codice, vengono mostrati i valori delle metriche di valutazione e la confusion matrix per ogni modello addestrato precedentemente. Alla fine, bisogna sceglierne uno.

In [7]:
for i in range(totFold):
    print(f'{i+1}) Fold')
    print('Precision: ', precisions[i])
    print('Recall: ', recalls[i])
    print('Accuracy: ', accuracies[i])
    print('F1-Score: ', f1_scores[i])
    print('Mean Squared Error: ', mse_scores[i])

    TN, FP, FN, TP = cms[i].ravel()
    print()
    print(f"True Positives (TP): {TP}")
    print(f"True Negatives (TN): {TN}")
    print(f"False Positives (FP): {FP}")
    print(f"False Negatives (FN): {FN}")
    print()

1) Fold
Precision:  1.0
Recall:  0.975
Accuracy:  0.9901315789473685
F1-Score:  0.9873417721518987
Mean Squared Error:  0.009868421052631578

True Positives (TP): 117
True Negatives (TN): 184
False Positives (FP): 0
False Negatives (FN): 3

2) Fold
Precision:  1.0
Recall:  1.0
Accuracy:  1.0
F1-Score:  1.0
Mean Squared Error:  0.0

True Positives (TP): 118
True Negatives (TN): 186
False Positives (FP): 0
False Negatives (FN): 0

3) Fold
Precision:  0.9919354838709677
Recall:  0.9919354838709677
Accuracy:  0.993421052631579
F1-Score:  0.9919354838709677
Mean Squared Error:  0.006578947368421052

True Positives (TP): 123
True Negatives (TN): 179
False Positives (FP): 1
False Negatives (FN): 1

4) Fold
Precision:  1.0
Recall:  0.9606299212598425
Accuracy:  0.9835526315789473
F1-Score:  0.9799196787148594
Mean Squared Error:  0.01644736842105263

True Positives (TP): 122
True Negatives (TN): 177
False Positives (FP): 0
False Negatives (FN): 5

5) Fold
Precision:  0.9809523809523809
Recall:

### 5.3) Visualizzazione grafico addestramento
Se viene utilizzato il set di validation per l'addestramento, rimuovere i commenti per la visualizzazione di "val_loss" e "val_accuracy"

In [ ]:
#plotting results
fig = plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(hist.history['loss'], color='teal', label='loss')
#plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=15)
plt.legend(loc="upper left")


plt.subplot(1, 2, 2)
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
#plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=15)
plt.legend(loc="upper left")
plt.show()

# 6) Salvataggio modello
In questo modo è possibile salvare il modello in memoria

In [ ]:
model.save(os.path.join('models','rete_1.h5'))

In [ ]:
#Salvataggio di tutti i modelli ottenuti dalla Fold Cross-Validation
for i in range(len(models)):
    models[i].save(os.path.join('models',f'{i+1}Fold_5.h5'))

# 7) Caricamento modello
Per utilizzare un modello vecchio utilizzato in precedenza, non è necessario rifare l'addestramento, ma basta semplicemente caricarlo

In [67]:
model = saving.load_model('models\\__rete5_d0_tilde.h5')

# 8) Processamento audio
A questo punto, si vuole far lavorare la rete su un audio. La sequenza è questa:

1) Caricamento audio e creazione degli spettrogrammi in scala di grigio
2) Filtraggio immagini con filtro Sobel, implementato nel file ImageFilter.py
3) Caricamento immagini in due dimensioni, ridimensionate (224, 224) e raggruppate in batch
4) Previsione da parte della rete

Alla fine si ottiene un array di percentuali, ogni valore si riferisce ad un immagine.

### 8.1) Definizione funzioni

In [83]:
#Questa funzione prende in ingresso un file wav e ne crea gli spettrogrammi in scala di grigio.
#Vengono selezionate solo le frequenze tra 5 kHz e 25 kHz.
#Le immagini fanno riferimento a frame di 600 ms, che scorrono di 200 ms. Per fare questa suddivisione temporale, viene creato un vettore che fa riferimento al tempo. In un ciclo vengono selezionati gli intervalli opportuni e viene memorizzata l'immagine in scala di grigio.
#I nomi dei file che vengono creati sono valori numerici in ordine crescente.
# Facendo questa suddivisione temporale, per una registrazione di un'ora si dovrebbero avere 17988 immagini, per una registrazione di 5 minuti 1498 immagini.

def getAndSaveSpectrogram(filename, outputPath, window_size=2048, overlap=1024):
    fs, signal = sci.io.wavfile.read(filename)
    step = window_size - overlap
    bins = np.arange(0, len(signal) - window_size + 1, step)
    window = np.hanning(window_size)
    spectrogram = []

    n_time_steps = int((len(signal) - window_size) / overlap) + 1

    time = ((n_time_steps + 1) * overlap) / fs
    print(f'{time} secondi')
    #print(str(n_time_steps) + ' time step')

    for start in bins:
        segment = signal[start:start + window_size] * window
        spectrum = np.fft.fft(segment, n=2048)[:window_size // 2]
        magnitude = np.abs(spectrum)
        magnitude_db = 20 * np.log10(magnitude)
        spectrogram.append(magnitude_db)

    spectrogram = np.array(spectrogram).T

    max_freq = 25000
    freq_bins = int(max_freq / (fs / window_size))
    min_freq = 5000
    freq_bins_min = int(min_freq / (fs / window_size))

    #375 step corrispondono ad un secondo
    oneSecStep = (n_time_steps / time) + 1
    lengthFrame = oneSecStep * 0.6
    print(oneSecStep, lengthFrame, oneSecStep * 0.2)
    lengthStep = 0.2
    n_frame = int(time / lengthStep) - 2

    arrayTime = [0]
    toNext = float((n_time_steps) / (n_frame+2))

    for k in range (1, n_frame+2):
        number = arrayTime[k-1] + toNext + (k%2)
        arrayTime.append(int(number))

    #print(arrayTime)

    for i in range(n_frame - 1):
        print(i + 1, n_frame)
        frame = spectrogram[freq_bins_min:freq_bins, arrayTime[i]:arrayTime[i + 3]]
        plt.gray()
        plt.rcParams['axes.grid'] = False
        plt.rcParams['image.origin'] = 'lower'
        plt.rcParams['image.aspect'] = 'auto'
        plt.axis('off')

        plt.imshow(frame)
        outputFile = f'{outputPath}\\{i+1}.png'
        plt.savefig(outputFile, bbox_inches='tight', pad_inches=0)
        plt.close()

    frame = spectrogram[freq_bins_min:freq_bins, arrayTime[n_frame - 1]:n_time_steps - 1]
    plt.gray()
    plt.rcParams['axes.grid'] = False
    plt.rcParams['image.origin'] = 'lower'
    plt.rcParams['image.aspect'] = 'auto'
    plt.axis('off')

    plt.imshow(frame)
    outputFile = f'{outputPath}\\{n_frame}.png'
    if n_frame==0:
        outputFile = f'{outputPath}\\1.png'

    plt.savefig(outputFile, bbox_inches='tight', pad_inches=0)
    plt.close()


#Questa funzione serve per eliminare tutti i file all'interno di una cartella
def delete_all_files_in_directory(directory_path):
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
        except Exception as e:
            print(f"Error deleting file {file_path}. Reason: {e}")


#Una volta creati gli spettrogrammi, questa funzione può essere utilizzata per far predirre alla rete le immagini.
#Se devono ancora essere filtrate tramite filtro Sobel, settare a True il parametro "toFilter". Se si hanno già gli spettrogrammi filtrati, è sufficiente richiamare questa funzione con il parametro toFilter settato a False. Questo permette di non dover creare ogni volta gli spettrogrammi se si vuole lavorare più volte con la stessa registrazione.
#Le immagini vengono raggruppate in batch, perchè in questo modo la rete è più veloce piuttosto che analizzare le immagini una ad una.
#Dopodichè, viene utilizzata la rete neurale per effettuare la previsione delle immagini.
#Questa funzione restituisce il risultato della previsione, ovvero un vettore di percentuali

def predictOnSpectrogramsBatches(pathSpectrograms, model, toFilter=False):

    if toFilter==True:
        filter.sobelFilterDirectory(pathSpectrograms, pathSpectrograms)

    spectrogramList = []
    spectrogram_list_os = os.listdir(pathSpectrograms)

    for i in range(len(spectrogram_list_os)):
        filePath = os.path.join(pathSpectrograms, f'{i + 1}.png')
        image = Image.open(filePath).convert('L')  # Convert to grayscale
        image = image.resize((224, 224))  # Resize if needed
        image = np.array(image).transpose()  # Convert to NumPy array
        spectrogramList.append((np.expand_dims(image, 0)).transpose())
        print(image.shape)

    spectrogramList = np.array(spectrogramList)/255.0

    BATCH_SIZE = 128
    indices = list(range(BATCH_SIZE, len(spectrogramList), BATCH_SIZE))

    # Split the array
    split_arrays = np.split(spectrogramList, indices)

    result = []
    count = 0
    # Display the split arrays
    for sub_array in split_arrays:
        yApp = model.predict(sub_array)
        for k in range(len(yApp)):  #extend
            result.append(yApp[k])
        count += len(yApp)
        print(str(count), 'done')

    return result



#Funzione utilizzata per effettuare tutti i passaggi descritti nell'elenco numerato. Gli spettrogrammi vengono memorizzati tutti all'interno della cartella "Analize"
def predictWave(filename, model):
    pathSpectrograms = 'Analize'
    delete_all_files_in_directory(pathSpectrograms)
    getAndSaveSpectrogram(filename, pathSpectrograms)
    return predictOnSpectrogramsBatches(pathSpectrograms, model, True)





### 8.2) Avvio previsione
Nel caso in cui gli spettrogrammi sono stati già fatti, eseguire "predictOnSpectrogramsBatches" invece di "predictWave"

In [ ]:
filePath = 'RegFatte\\20211120_174159_192.wav'
result = predictWave(filePath, model)
#result = predictOnSpectrogramsBatches('Analize', model)

### 8.3) Visualizzazione risultati
In questo blocco di codice vengono visualizzate le percentuali di appartenenza alla classe dei delfini.
Vengono riportati sia il numero dell'immagine a cui si fa riferimento che l'istante temporale di partenza.

In [75]:
for i in range(len(result)):
    seconds = int(i / 5 % 60)
    if seconds < 10:
        seconds = '0' + str(seconds)
    else:
        seconds = str(seconds)

    seconds += f':{(i%5)*2}0'

    #if result[i] > 0.5:
    print(i + 1, str(int(i / 5 / 60)) + ':' + seconds, '- prob: ' + str(int(result[i] * 1000) / 10))

1 0:00:00 - prob: 0.0
2 0:00:20 - prob: 0.0
3 0:00:40 - prob: 0.0
4 0:00:60 - prob: 0.0
5 0:00:80 - prob: 0.0
6 0:01:00 - prob: 0.0
7 0:01:20 - prob: 0.0
8 0:01:40 - prob: 0.0
9 0:01:60 - prob: 0.0
10 0:01:80 - prob: 0.0
11 0:02:00 - prob: 0.0
12 0:02:20 - prob: 0.0
13 0:02:40 - prob: 0.0
14 0:02:60 - prob: 0.0
15 0:02:80 - prob: 100.0
16 0:03:00 - prob: 100.0
17 0:03:20 - prob: 100.0
18 0:03:40 - prob: 99.9
19 0:03:60 - prob: 0.0
20 0:03:80 - prob: 0.0
21 0:04:00 - prob: 0.0
22 0:04:20 - prob: 0.0
23 0:04:40 - prob: 0.0
24 0:04:60 - prob: 0.0
25 0:04:80 - prob: 0.0
26 0:05:00 - prob: 0.0
27 0:05:20 - prob: 0.0
28 0:05:40 - prob: 0.0
29 0:05:60 - prob: 0.0
30 0:05:80 - prob: 21.2
31 0:06:00 - prob: 0.0
32 0:06:20 - prob: 0.0
33 0:06:40 - prob: 0.0
34 0:06:60 - prob: 0.0
35 0:06:80 - prob: 0.0
36 0:07:00 - prob: 0.0
37 0:07:20 - prob: 0.0
38 0:07:40 - prob: 0.0
39 0:07:60 - prob: 0.0
40 0:07:80 - prob: 0.0
41 0:08:00 - prob: 0.0
42 0:08:20 - prob: 0.0
43 0:08:40 - prob: 0.0
44 0:08:60 -

C:\Users\nicol\AppData\Local\Temp\ipykernel_7212\1571403754.py:10: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(i + 1, str(int(i / 5 / 60)) + ':' + seconds, '- prob: ' + str(int(result[i] * 1000) / 10))


# 9) Interpretazione dei risultati
Questo blocco di codice definisce la funzione per stampare le etichette, ovvero dichiarare l'inizio e la fine di uno o più fischi consecutivi all'interno della registrazione, salvandole in un file di testo.
Per il posizionamento delle etichette si fa una precisazione, aggiungere 1 all'indice significa aggiungere 200 ms.

In [84]:
def makeAndSaveLabels(result, filename):

    etichette = []
    indexList = []
    indexStart = -1
    indexStop = -1
    hold = 0

    for i in range(len(result)):
        if not(hold == 0):
            if (result[i] > 0.5 and hold==2) or (result[i] > 0.70 and hold==1):
                indexList.append(i)
            else:
                indexStop = i+1.5*(int(i<len(result)-1)) #non sarà mai -1
                #scrivi in etichetta    indexStart  indexStop
                etichette.append(f'{(indexStart+1)/5}\t{indexStop/5}\tW')
                hold=0

        else:

            next = i + 1 - int(i==(len(result)-1))

            if result[i] > 0.7:
                if(result[abs(i-1)]>0.7 or result[next]>0.7):
                    indexList.append(i)
                    indexStart = i
                    hold = 1

                if result[i] > 0.9:
                    if result[abs(i-1)]>0.5 or result[next]>0.5 :
                        indexList.append(i)
                        indexStart = i
                        hold = 2

                if result[i] > 0.995:
                    indexList.append(i)
                    indexStart = i
                    hold = 2


    # Write to the file
    with open(filename, 'w') as file:
        for item in etichette:
            file.write('%s\n' % item)

    return etichette

In [80]:
filename = 'Registrazioni_Tesi\\_.txt'
etichette = makeAndSaveLabels(result, filename)

print(len(etichette))

16


# 10) Processamento audio e stampaggio etichette singolo file
In questa parte di codice viene semplicemente definita la funzione da richiamare nel caso in cui si voglia processare un file audio e stamparne le etichette in automatico

In [0]:
def predictWaveAndSaveLabels(fileWave, model, outputFile):
    pathSpectrograms = 'Analize'
    delete_all_files_in_directory(pathSpectrograms)
    getAndSaveSpectrogram(fileWave, pathSpectrograms)
    result = predictOnSpectrogramsBatches(pathSpectrograms, model, True)
    makeAndSaveLabels(result, outputFile)

# 11) Processamento audio e stampaggio etichette di tutti i file Wave in una cartella


In [ ]:
pathWaves = 'Registrazioni_Tesi'
listWaves = []
listWaves = os.listdir(pathWaves)
print(listWaves)

for recName in listWaves:

    if(recName.split('.')[1] != 'wav'):
        continue

    delete_all_files_in_directory('Analize')
    fileWave = os.path.join(pathWaves, recName)
    print('\n\n\n\n', recName)

    try:
        recName = recName.split('.')[0]
        fileLabels = os.path.join(pathWaves, recName) + '.txt'

        predictWaveAndSaveLabels(fileWave, model, fileLabels)

        #gc.collect()

    except Exception as e:
            print(f"Error! Reason: {e}")
